In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings

In [36]:
df = pd.read_csv("/Users/elaine/Desktop/side_project/stock/stock_company.csv",usecols=["main_stock_symbol","sub_stock_symbol","Company_E"])
df

,main_stock_symbol,sub_stock_symbol,Company_E
0,50,2002,CHINA STEEL CORPORATION
1,50,2412,"Chunghwa Telecom Co., Ltd."
2,50,2308,"DELTA ELECTRONICS, INC."
3,50,2317,"HON HAI PRECISION INDUSTRY CO., LTD."
4,50,2454,MEDIATEK INC.
...,...,...,...
80,56,2006,NaN
81,56,2809,NaN
82,56,3665,NaN
83,56,6271,"TONG HSING ELECTRONIC INDUSTRIES, LTD."


In [37]:
df = df[df["Company_E"].notna()]

In [38]:
#precise

url = "https://api.patentsview.org/patents/query"
json_arr_list = []

Company_E = df["Company_E"].to_list()

for i in Company_E:

    data = {"q":{ "_and":[{"assignee_organization":i},{"_gte":{"app_date":"2012-01-01"}}]}, "f":["patent_number","assignee_organization","patent_abstract","patent_date","inventor_first_name","inventor_last_name","cpc_subgroup_id","app_date"],"o":{"page":1,"per_page":100000}}
    r = requests.post(url,  json = data)

    ans = json.loads(r.text)
    json_arr_list.append(ans)
    
with open("/Users/elaine/Desktop/side_project/stock/stock_data.json", "w") as outfile: #save file
    json.dump(json_arr_list, outfile)

In [46]:
with open("/Users/elaine/Desktop/side_project/stock/stock_data.json","r") as f:  #change file
     data = json.loads(f.read())
result = pd.json_normalize(data, record_path = ["patents"])
result

,patent_number,patent_abstract,patent_date,inventors,assignees,applications,cpcs
0,10047418,A method for manufacturing a high-strength and...,2018-08-14,"[{'inventor_first_name': 'Kuo-Cheng', 'invento...",[{'assignee_organization': 'CHINA STEEL CORPOR...,"[{'app_date': '2015-07-10', 'app_id': '14/7963...","[{'cpc_subgroup_id': 'C21D6/005'}, {'cpc_subgr..."
1,8556185,The present invention relates to an RFID tag h...,2013-10-15,"[{'inventor_first_name': 'Sung-Lin', 'inventor...",[{'assignee_organization': 'CHINA STEEL CORPOR...,"[{'app_date': '2013-04-17', 'app_id': '13/8647...","[{'cpc_subgroup_id': 'G06K19/07788'}, {'cpc_su..."
2,8561912,The present invention relates to an RFID tag h...,2013-10-22,"[{'inventor_first_name': 'Sung-Lin', 'inventor...",[{'assignee_organization': 'CHINA STEEL CORPOR...,"[{'app_date': '2013-04-17', 'app_id': '13/8646...","[{'cpc_subgroup_id': 'G06K19/07788'}, {'cpc_su..."
3,9048388,A multi-layer thermoelectric module and a fabr...,2015-06-02,"[{'inventor_first_name': 'Tung-Han', 'inventor...",[{'assignee_organization': 'CHINA STEEL CORPOR...,"[{'app_date': '2013-07-08', 'app_id': '13/9371...","[{'cpc_subgroup_id': 'H01L35/325'}, {'cpc_subg..."
4,9096810,A method for producing a bio-coal includes the...,2015-08-04,"[{'inventor_first_name': 'Huang-Chih', 'invent...",[{'assignee_organization': 'CHINA STEEL CORPOR...,"[{'app_date': '2013-01-23', 'app_id': '13/7480...","[{'cpc_subgroup_id': 'C10L5/08'}, {'cpc_subgro..."
...,...,...,...,...,...,...,...
43675,D948525,None,2022-04-12,"[{'inventor_first_name': 'Hung-Chi', 'inventor...",[{'assignee_organization': 'Acer Incorporated'...,"[{'app_date': '2020-12-11', 'app_id': '2976171...",[{'cpc_subgroup_id': None}]
43676,D955383,None,2022-06-21,"[{'inventor_first_name': 'Chun-Te', 'inventor_...",[{'assignee_organization': 'Acer Incorporated'...,"[{'app_date': '2020-04-10', 'app_id': '2973096...",[{'cpc_subgroup_id': None}]
43677,D955400,None,2022-06-21,"[{'inventor_first_name': 'Wei-Chang', 'invento...",[{'assignee_organization': 'Acer Incorporated'...,"[{'app_date': '2020-12-14', 'app_id': '2976191...",[{'cpc_subgroup_id': None}]
43678,D962224,None,2022-08-30,"[{'inventor_first_name': 'Jung-Wei', 'inventor...",[{'assignee_organization': 'Acer Incorporated'...,"[{'app_date': '2021-03-08', 'app_id': '2977325...",[{'cpc_subgroup_id': None}]


In [47]:
result.rename(columns = {"applications":"app_date"},inplace = True)
result

,patent_number,patent_abstract,patent_date,inventors,assignees,app_date,cpcs
0,10047418,A method for manufacturing a high-strength and...,2018-08-14,"[{'inventor_first_name': 'Kuo-Cheng', 'invento...",[{'assignee_organization': 'CHINA STEEL CORPOR...,"[{'app_date': '2015-07-10', 'app_id': '14/7963...","[{'cpc_subgroup_id': 'C21D6/005'}, {'cpc_subgr..."
1,8556185,The present invention relates to an RFID tag h...,2013-10-15,"[{'inventor_first_name': 'Sung-Lin', 'inventor...",[{'assignee_organization': 'CHINA STEEL CORPOR...,"[{'app_date': '2013-04-17', 'app_id': '13/8647...","[{'cpc_subgroup_id': 'G06K19/07788'}, {'cpc_su..."
2,8561912,The present invention relates to an RFID tag h...,2013-10-22,"[{'inventor_first_name': 'Sung-Lin', 'inventor...",[{'assignee_organization': 'CHINA STEEL CORPOR...,"[{'app_date': '2013-04-17', 'app_id': '13/8646...","[{'cpc_subgroup_id': 'G06K19/07788'}, {'cpc_su..."
3,9048388,A multi-layer thermoelectric module and a fabr...,2015-06-02,"[{'inventor_first_name': 'Tung-Han', 'inventor...",[{'assignee_organization': 'CHINA STEEL CORPOR...,"[{'app_date': '2013-07-08', 'app_id': '13/9371...","[{'cpc_subgroup_id': 'H01L35/325'}, {'cpc_subg..."
4,9096810,A method for producing a bio-coal includes the...,2015-08-04,"[{'inventor_first_name': 'Huang-Chih', 'invent...",[{'assignee_organization': 'CHINA STEEL CORPOR...,"[{'app_date': '2013-01-23', 'app_id': '13/7480...","[{'cpc_subgroup_id': 'C10L5/08'}, {'cpc_subgro..."
...,...,...,...,...,...,...,...
43675,D948525,None,2022-04-12,"[{'inventor_first_name': 'Hung-Chi', 'inventor...",[{'assignee_organization': 'Acer Incorporated'...,"[{'app_date': '2020-12-11', 'app_id': '2976171...",[{'cpc_subgroup_id': None}]
43676,D955383,None,2022-06-21,"[{'inventor_first_name': 'Chun-Te', 'inventor_...",[{'assignee_organization': 'Acer Incorporated'...,"[{'app_date': '2020-04-10', 'app_id': '2973096...",[{'cpc_subgroup_id': None}]
43677,D955400,None,2022-06-21,"[{'inventor_first_name': 'Wei-Chang', 'invento...",[{'assignee_organization': 'Acer Incorporated'...,"[{'app_date': '2020-12-14', 'app_id': '2976191...",[{'cpc_subgroup_id': None}]
43678,D962224,None,2022-08-30,"[{'inventor_first_name': 'Jung-Wei', 'inventor...",[{'assignee_organization': 'Acer Incorporated'...,"[{'app_date': '2021-03-08', 'app_id': '2977325...",[{'cpc_subgroup_id': None}]


In [48]:
inventors = []
for inven in result["inventors"]:
    inventor_str = ""
    for names in inven:
        first_name = names["inventor_first_name"]
        last_name = names["inventor_last_name"]
        inventor_str = inventor_str + (first_name  + "," + last_name) + ";"
    inventors.append(inventor_str)

result["inventors"] = inventors
result["inventors"] = result["inventors"].str[:-1]
result

,patent_number,patent_abstract,patent_date,inventors,assignees,app_date,cpcs
0,10047418,A method for manufacturing a high-strength and...,2018-08-14,"Kuo-Cheng,Yang;Lung-Jen,Chiang;Ming-Chin,Tsai;...",[{'assignee_organization': 'CHINA STEEL CORPOR...,"[{'app_date': '2015-07-10', 'app_id': '14/7963...","[{'cpc_subgroup_id': 'C21D6/005'}, {'cpc_subgr..."
1,8556185,The present invention relates to an RFID tag h...,2013-10-15,"Sung-Lin,Chen;Yi-An,Wu",[{'assignee_organization': 'CHINA STEEL CORPOR...,"[{'app_date': '2013-04-17', 'app_id': '13/8647...","[{'cpc_subgroup_id': 'G06K19/07788'}, {'cpc_su..."
2,8561912,The present invention relates to an RFID tag h...,2013-10-22,"Sung-Lin,Chen;Yi-An,Wu",[{'assignee_organization': 'CHINA STEEL CORPOR...,"[{'app_date': '2013-04-17', 'app_id': '13/8646...","[{'cpc_subgroup_id': 'G06K19/07788'}, {'cpc_su..."
3,9048388,A multi-layer thermoelectric module and a fabr...,2015-06-02,"Tung-Han,Chuang;Jing-Yi,Huang;Jenn-Dong,Hwang;...",[{'assignee_organization': 'CHINA STEEL CORPOR...,"[{'app_date': '2013-07-08', 'app_id': '13/9371...","[{'cpc_subgroup_id': 'H01L35/325'}, {'cpc_subg..."
4,9096810,A method for producing a bio-coal includes the...,2015-08-04,"Huang-Chih,Cheng;Yung-Ping,Chu;Der-Her,Wang;Ch...",[{'assignee_organization': 'CHINA STEEL CORPOR...,"[{'app_date': '2013-01-23', 'app_id': '13/7480...","[{'cpc_subgroup_id': 'C10L5/08'}, {'cpc_subgro..."
...,...,...,...,...,...,...,...
43675,D948525,None,2022-04-12,"Hung-Chi,Chen;Cheng-Han,Lin",[{'assignee_organization': 'Acer Incorporated'...,"[{'app_date': '2020-12-11', 'app_id': '2976171...",[{'cpc_subgroup_id': None}]
43676,D955383,None,2022-06-21,"Chun-Te,Lin;Tsun-Chih,Yang",[{'assignee_organization': 'Acer Incorporated'...,"[{'app_date': '2020-04-10', 'app_id': '2973096...",[{'cpc_subgroup_id': None}]
43677,D955400,None,2022-06-21,"Wei-Chang,Chen;Jung-Wei,Tsao",[{'assignee_organization': 'Acer Incorporated'...,"[{'app_date': '2020-12-14', 'app_id': '2976191...",[{'cpc_subgroup_id': None}]
43678,D962224,None,2022-08-30,"Jung-Wei,Tsao",[{'assignee_organization': 'Acer Incorporated'...,"[{'app_date': '2021-03-08', 'app_id': '2977325...",[{'cpc_subgroup_id': None}]


In [49]:
assignees = []
for orgs in result["assignees"]:
    orgs_list = []
    for org in orgs:
        assignee_organization = org["assignee_organization"]
        if assignee_organization == None:
            continue
        orgs_list.append(assignee_organization)
    assignees.append(orgs_list)
    
assignees_list = []
for assi in assignees:
    assi_join = ";".join(assi)
    assignees_list.append(assi_join)

result["assignees"] = assignees_list
result

,patent_number,patent_abstract,patent_date,inventors,assignees,app_date,cpcs
0,10047418,A method for manufacturing a high-strength and...,2018-08-14,"Kuo-Cheng,Yang;Lung-Jen,Chiang;Ming-Chin,Tsai;...",CHINA STEEL CORPORATION,"[{'app_date': '2015-07-10', 'app_id': '14/7963...","[{'cpc_subgroup_id': 'C21D6/005'}, {'cpc_subgr..."
1,8556185,The present invention relates to an RFID tag h...,2013-10-15,"Sung-Lin,Chen;Yi-An,Wu",CHINA STEEL CORPORATION,"[{'app_date': '2013-04-17', 'app_id': '13/8647...","[{'cpc_subgroup_id': 'G06K19/07788'}, {'cpc_su..."
2,8561912,The present invention relates to an RFID tag h...,2013-10-22,"Sung-Lin,Chen;Yi-An,Wu",CHINA STEEL CORPORATION,"[{'app_date': '2013-04-17', 'app_id': '13/8646...","[{'cpc_subgroup_id': 'G06K19/07788'}, {'cpc_su..."
3,9048388,A multi-layer thermoelectric module and a fabr...,2015-06-02,"Tung-Han,Chuang;Jing-Yi,Huang;Jenn-Dong,Hwang;...",CHINA STEEL CORPORATION,"[{'app_date': '2013-07-08', 'app_id': '13/9371...","[{'cpc_subgroup_id': 'H01L35/325'}, {'cpc_subg..."
4,9096810,A method for producing a bio-coal includes the...,2015-08-04,"Huang-Chih,Cheng;Yung-Ping,Chu;Der-Her,Wang;Ch...",CHINA STEEL CORPORATION,"[{'app_date': '2013-01-23', 'app_id': '13/7480...","[{'cpc_subgroup_id': 'C10L5/08'}, {'cpc_subgro..."
...,...,...,...,...,...,...,...
43675,D948525,None,2022-04-12,"Hung-Chi,Chen;Cheng-Han,Lin",Acer Incorporated,"[{'app_date': '2020-12-11', 'app_id': '2976171...",[{'cpc_subgroup_id': None}]
43676,D955383,None,2022-06-21,"Chun-Te,Lin;Tsun-Chih,Yang",Acer Incorporated,"[{'app_date': '2020-04-10', 'app_id': '2973096...",[{'cpc_subgroup_id': None}]
43677,D955400,None,2022-06-21,"Wei-Chang,Chen;Jung-Wei,Tsao",Acer Incorporated,"[{'app_date': '2020-12-14', 'app_id': '2976191...",[{'cpc_subgroup_id': None}]
43678,D962224,None,2022-08-30,"Jung-Wei,Tsao",Acer Incorporated,"[{'app_date': '2021-03-08', 'app_id': '2977325...",[{'cpc_subgroup_id': None}]


In [50]:
app_dates = []
for applications in result["app_date"]:
    
    dates = list(map(lambda d: d['app_date'], applications))
    for date in dates:
        app_dates.append(date)

result["app_date"] = app_dates
result

,patent_number,patent_abstract,patent_date,inventors,assignees,app_date,cpcs
0,10047418,A method for manufacturing a high-strength and...,2018-08-14,"Kuo-Cheng,Yang;Lung-Jen,Chiang;Ming-Chin,Tsai;...",CHINA STEEL CORPORATION,2015-07-10,"[{'cpc_subgroup_id': 'C21D6/005'}, {'cpc_subgr..."
1,8556185,The present invention relates to an RFID tag h...,2013-10-15,"Sung-Lin,Chen;Yi-An,Wu",CHINA STEEL CORPORATION,2013-04-17,"[{'cpc_subgroup_id': 'G06K19/07788'}, {'cpc_su..."
2,8561912,The present invention relates to an RFID tag h...,2013-10-22,"Sung-Lin,Chen;Yi-An,Wu",CHINA STEEL CORPORATION,2013-04-17,"[{'cpc_subgroup_id': 'G06K19/07788'}, {'cpc_su..."
3,9048388,A multi-layer thermoelectric module and a fabr...,2015-06-02,"Tung-Han,Chuang;Jing-Yi,Huang;Jenn-Dong,Hwang;...",CHINA STEEL CORPORATION,2013-07-08,"[{'cpc_subgroup_id': 'H01L35/325'}, {'cpc_subg..."
4,9096810,A method for producing a bio-coal includes the...,2015-08-04,"Huang-Chih,Cheng;Yung-Ping,Chu;Der-Her,Wang;Ch...",CHINA STEEL CORPORATION,2013-01-23,"[{'cpc_subgroup_id': 'C10L5/08'}, {'cpc_subgro..."
...,...,...,...,...,...,...,...
43675,D948525,None,2022-04-12,"Hung-Chi,Chen;Cheng-Han,Lin",Acer Incorporated,2020-12-11,[{'cpc_subgroup_id': None}]
43676,D955383,None,2022-06-21,"Chun-Te,Lin;Tsun-Chih,Yang",Acer Incorporated,2020-04-10,[{'cpc_subgroup_id': None}]
43677,D955400,None,2022-06-21,"Wei-Chang,Chen;Jung-Wei,Tsao",Acer Incorporated,2020-12-14,[{'cpc_subgroup_id': None}]
43678,D962224,None,2022-08-30,"Jung-Wei,Tsao",Acer Incorporated,2021-03-08,[{'cpc_subgroup_id': None}]


In [51]:
cpcs = []
for cpc_subgroup_id in result["cpcs"]:   
    
    cpc_set = []
    for sub_cpc in cpc_subgroup_id:
        cpc = sub_cpc["cpc_subgroup_id"] 
        
        if cpc == None:        
            continue
        ch = "/"
        cpc = cpc.split(ch,1)[0] 
        cpc_set.append(cpc)    
    cpc_set = list(set(cpc_set))
    cpcs.append(cpc_set)

cpcs_list = []
for cpc_str in cpcs:
    cpc_join = ",".join(cpc_str)
    cpcs_list.append(cpc_join)

result["cpcs"] = cpcs_list
result

,patent_number,patent_abstract,patent_date,inventors,assignees,app_date,cpcs
0,10047418,A method for manufacturing a high-strength and...,2018-08-14,"Kuo-Cheng,Yang;Lung-Jen,Chiang;Ming-Chin,Tsai;...",CHINA STEEL CORPORATION,2015-07-10,"C22C38,C21D6,C21D8,C21D2211"
1,8556185,The present invention relates to an RFID tag h...,2013-10-15,"Sung-Lin,Chen;Yi-An,Wu",CHINA STEEL CORPORATION,2013-04-17,"H01Q1,G06K19"
2,8561912,The present invention relates to an RFID tag h...,2013-10-22,"Sung-Lin,Chen;Yi-An,Wu",CHINA STEEL CORPORATION,2013-04-17,"H01Q1,G06K19"
3,9048388,A multi-layer thermoelectric module and a fabr...,2015-06-02,"Tung-Han,Chuang;Jing-Yi,Huang;Jenn-Dong,Hwang;...",CHINA STEEL CORPORATION,2013-07-08,H01L35
4,9096810,A method for producing a bio-coal includes the...,2015-08-04,"Huang-Chih,Cheng;Yung-Ping,Chu;Der-Her,Wang;Ch...",CHINA STEEL CORPORATION,2013-01-23,"C10L9,C10L5,Y02E50,C10L2290,C10L2250"
...,...,...,...,...,...,...,...
43675,D948525,None,2022-04-12,"Hung-Chi,Chen;Cheng-Han,Lin",Acer Incorporated,2020-12-11,
43676,D955383,None,2022-06-21,"Chun-Te,Lin;Tsun-Chih,Yang",Acer Incorporated,2020-04-10,
43677,D955400,None,2022-06-21,"Wei-Chang,Chen;Jung-Wei,Tsao",Acer Incorporated,2020-12-14,
43678,D962224,None,2022-08-30,"Jung-Wei,Tsao",Acer Incorporated,2021-03-08,


In [52]:
result.to_csv(f"/Users/elaine/Desktop/side_project/stock/uspto_api_final_result.csv",index = False)